In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("prices")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-05-05 2023-05-06


In [4]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

In [5]:
values

,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,dates
0,3.672504,87.000368,101.280403,386.350403,1.802185,509.000367,222.360555,1.47905,1.8025,1.70397,...,2.70255,0.739504,594.503597,108.750364,250.350363,18.221455,9001.203589,17.997551,321.999592,2023-05-05
1,3.672504,87.000368,101.280403,386.350403,1.802185,509.000367,222.340722,1.47905,1.8025,1.70397,...,2.70255,0.739504,594.503597,108.750364,250.350363,18.221455,9001.203589,17.997551,321.999592,2023-05-06


In [6]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("prices",data)
        except:
            print(ticker,"tiingo")
        sleep(1)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|█▏                                                                                                                                              | 4/491 [00:06<13:09,  1.62s/it]

market prices documents must be a non-empty list


 39%|███████████████████████████████████████████████████████▏                                                                                      | 191/491 [05:11<08:03,  1.61s/it]

If using all scalar values, you must pass an index
FRC tiingo


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌              | 441/491 [11:55<01:20,  1.61s/it]

market prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 491/491 [13:16<00:00,  1.62s/it]


In [7]:
data

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ticker
0,2023-05-05T00:00:00.000Z,186.23,187.8199,176.91,180.0,4224388,186.23,187.8199,176.91,180.0,4224388,0.0,1.0,ZTS


In [8]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 491/491 [00:42<00:00, 11.49it/s]


In [9]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
489,-0.505846,-0.184066,-0.114094,-0.114094,0.192173,ZION,Zions Bancorporation,Financials,Regional Banks
118,-0.439292,-0.136493,-0.142588,-0.142588,0.167574,CMA,Comerica,Financials,Diversified Banks
278,0.120534,0.113259,0.128070,0.128070,0.150440,LYV,Live Nation Entertainment,Communication Services,Movies & Entertainment
261,-0.435472,-0.205335,-0.083022,-0.083022,0.100784,KEY,KeyCorp,Financials,Regional Banks
440,-0.335291,-0.134018,-0.089784,-0.089784,0.094620,TFC,Truist,Financials,Regional Banks
32,-0.139791,0.066839,0.013185,0.013185,0.078388,AIG,American International Group,Financials,Property & Casualty Insurance
283,-0.575419,-0.119691,-0.042017,-0.042017,0.075472,LUMN,Lumen Technologies,Communication Services,Alternative Carriers
190,-0.226947,-0.056061,-0.034857,-0.034857,0.072289,FITB,Fifth Third Bank,Financials,Regional Banks
146,-0.495035,-0.213260,-0.039136,-0.039136,0.070677,DISH,Dish Network,Communication Services,Cable & Satellite
105,-0.185398,-0.004809,0.022911,0.022911,0.069966,CI,Cigna,Health Care,Managed Health Care


In [10]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Staples,0.055066,0.034295,-0.005527,-0.005527,0.008260
Utilities,-0.000286,0.025513,-0.000863,-0.000863,0.007153
Health Care,0.022447,0.013645,-0.013952,-0.013952,0.008587
Real Estate,-0.003886,0.012403,-0.002807,-0.002807,0.015911
Consumer Discretionary,0.090326,0.000520,-0.010044,-0.010044,0.020054
Industrials,0.049756,-0.006603,-0.004274,-0.004274,0.017772
Materials,-0.002154,-0.030022,-0.013667,-0.013667,0.016678
Communication Services,0.054158,-0.031218,-0.017274,-0.026840,0.027556
Financials,-0.107884,-0.046831,-0.034189,-0.034189,0.032789
